In [15]:
###########import packages##########
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn.linear_model import BayesianRidge
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import ExtraTreeClassifier
from sklearn import linear_model
import lightgbm
import catboost
import xgboost
import shap
import seaborn as sns
from catboost import *
%matplotlib
###########loading data##########
fdata=pd.read_csv('database_filled_MFECD.csv',encoding="gbk")
raw_data=fdata.loc[:,[
                      'Relative Atomic Mass',#0
                      'Atomic Number',#1
                      'Ionization Potential',#2
                      'Electronegativity',#3
                      'Number of d electrons',#4
                      'ZIF or MOF Derived',#5
                      'Hard or Soft Templated',#6
                      'Graphene/Carbon Nanosheets or other 2D Structures',#7
                      'Polymer Derived',#8
                      'Carbon Nanofiber/Nanotubes',#9
                      'Carbon Black Derived',#10
                      'Biomass or other Organic Derived',#11  
                      'Main Transition Metal Content (wt. %)',#12
                      'Nitrogen Cotent (wt. %)',#13
                      'Metal-N Coordination Number (XAS)',#14    
                      'Pyridinic N Ratio',#15
                      'Pyrrolic N Ratio',#16
                      'Graphitic N Ratio',#17
                      'Oxidized N Ratio',#18
                      'Raman ID/IG Ratio',#19
                      'BET Surface Area (m2/g)',#20
                      'Acid Wash',#21
                      'Pyrolysis Temperature (°C)',#22
                      'Pyrolysis Time (h)',#23
                      'Rising Rate (°C min-1)',#24
                      'Flow Cell/H-type Cell',#25
                      'Electrolyte Concentration (M)',#26
                      'Catalyst Loading (mg cm-2)',#27
                      'Nafion Membrane Thickness (μm)',#28
                      'Carbon Paper/Glassy Carbon',#29
                      'Electrode Area cm2',#30
                      'Electrolyte pH',#31
                      'CLCD@MFE'#32
                        ]]
###########train test splitting##########
raw_param=raw_data.iloc[:,0:33]
print('ready')


Using matplotlib backend: Qt5Agg
ready


In [16]:
mat_sp=raw_data.corr(method='spearman')#kendall #pearson
mat_kd=raw_data.corr(method='kendall')
mat_ps=raw_data.corr(method='pearson')

In [20]:
f, ax= plt.subplots(figsize = (14, 10))
sns.set(font_scale=0.5)
sns.heatmap(mat_ps, 
                annot=True, vmax=1,vmin = 0, xticklabels= True, yticklabels= True, square=True, cmap="gist_heat_r")

<AxesSubplot:>

In [21]:
CDMFE_MCL=fdata.loc[:,['CLCD@MFE']]

In [22]:
def summarize(list1):
    total=0
    for ele in range(0, len(list1)):
        total = total + list1[ele]
    return total
from sklearn.decomposition import PCA

In [23]:
pca_3 = PCA(n_components=3)
chem_3 = pca_3.fit_transform(raw_data)     
print(pca_3.explained_variance_ratio_)

pca_2 = PCA(n_components=2)
chem_2 = pca_2.fit_transform(raw_data)     
print(pca_2.explained_variance_ratio_)

print(summarize(pca_3.explained_variance_ratio_))

print(summarize(pca_2.explained_variance_ratio_))

[0.83525485 0.14887899 0.00601671]
[0.83525485 0.14887899]
0.9901505452810624
0.9841338353071636


In [24]:
from mpl_toolkits.mplot3d import Axes3D
def plot_3d_PCA(dataset,classification_result):
    cl=classification_result.values[:,0]
    colors = ['red', 'blue','green']
    # 根据主成分分析结果绘制散点图
    fig=plt.figure()
    ax=Axes3D(fig)
    for i in range(len(colors)):
        x = dataset[:, 0][cl == i]
        y = dataset[:, 1][cl == i]
        z = dataset[:, 2][cl == i]
        ax.scatter(x,y,z,c=colors[i])
        ax.set_zlabel('Third Principal Component',fontsize='xx-large')
    # 设置图例，0-9用不同颜色表示
    if max(cl)>1:
        plt.legend(np.array([0,1,2]), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,fontsize='xx-large')
    else:
        plt.legend(np.array([0,1]), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,fontsize='xx-large')
    # 设置坐标标签
    plt.xlabel('First Principal Component',fontsize='xx-large')
    plt.ylabel('Second Principal Component',fontsize='xx-large')
    # 设置标题
    plt.title("PCA Scatter Plot 3D",fontsize='xx-large')
    plt.show()
def plot_2d_PCA(dataset,classification_result):
    cl=classification_result.values[:,0]
    colors = ['red', 'blue','green']
    # 根据主成分分析结果绘制散点图
    fig=plt.figure()
    for i in range(len(colors)):
        x = dataset[:, 0][cl == i]
        y = dataset[:, 1][cl == i]
        plt.scatter(x,y,c=colors[i])
    # 设置图例，0-9用不同颜色表示
    if max(cl)>1:
        plt.legend(np.array([0,1,2]), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,fontsize='xx-large')
    else:
        plt.legend(np.array([0,1]), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,fontsize='xx-large')
    # 设置坐标标签
    plt.xlabel('First Principal Component',fontsize='xx-large')
    plt.ylabel('Second Principal Component',fontsize='xx-large')
    # 设置标题
    plt.title("PCA Scatter Plot 2D",fontsize='xx-large')
    # 显示图形
    plt.show()

In [25]:
plot_3d_PCA(chem_3,CDMFE_MCL)

Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.


In [26]:
plot_2d_PCA(chem_2,CDMFE_MCL)